### UTFPR - UNIVERSIDADE TECNOLÓGICA FEDERAL DO PARANÁ
### DAELT - DEPARTAMENTO ACADÊMICO DE ELETROTÉCNICA

**Disciplina**: ET7GF - Sinais e Sistemas 2

**Professor**: Glauber Brante

**Aluno**: Douglas Dangui Pedroso

---

## Atividade de Simulação 11:
**Projeto de Filtros: Efeitos em Áudio** 

---

### Funções: ###

In [5]:
from numpy import *
from matplotlib.pyplot import *
from scipy.signal import *
from pyaudio import *
from scipy.io.wavfile import *

def impulso(n):
    d = 1*(n==0)
    return d

def degrau(n):
    d = 1*(n>=0)
    return d

def retangulo(n,N):
    d = 1*(n>=-N) - 1*(n>N)
    return d

def energia_potencia(x,i,f):
    e = sum(x*x)
    p = e/(f-i+1)
    return e,p

#  Função para atraso ou adiantamento de sinais com variável
#  independente invertida ou não
def meudeslocamento(x,nx,n0,invertido):
    
    if invertido == 'sim':
        if n0 >= 0:       
          ny = arange(nx[0]-abs(n0),nx[size(nx)-1]+1)
        else:
          ny = arange(nx[0],nx[size(nx)-1]+abs(n0)+1)                       
        if n0 >= 0:
            y = append(zeros(abs(n0)),x)
        else:
            y = append(x,zeros(abs(n0)))
    else:
        if n0 >= 0:       
          ny = arange(nx[0]-abs(n0),nx[size(nx)-1]+1)
        else:
          ny = arange(nx[0],nx[size(nx)-1]+abs(n0)+1) 
                          
        if n0 >= 0:
          y = append(x,zeros(abs(n0))) 
        else:
          y = append(zeros(abs(n0)),x)             
    return ny,y

def plot_entrada_saida(nx,x,ny,y):
    clf()
    subplot(2,1,1)
    stem(nx, x)
    grid(True)
    xlabel('n', fontsize=18)
    ylabel('Entrada x[n]', fontsize=18)
     
    subplot(2,1,2)
    stem(ny, y)
    grid(True)
    xlabel('n', fontsize=18)
    ylabel('Saída y[n]', fontsize=18)
    
def MM(nx,x,ordem,coef):
    nx = nx
    x = x
    ordem = ordem
    y = coef[0]*x
    for i in range(1,ordem+1):
        [ny,yd] = meudeslocamento(x,nx,-i,'não')
        y = append(y,0) + coef[i]*yd
        
    return ny,y

def uniform_midtread_quantizer(x,n):
    x = copy(x)
    n = copy(n)
    m = size(x,axis=0)
    Q = (1-(-1))/(2**n)
    for i in range(0,m):
        if x[i] >= (1-Q):
            x[i] = 1
        elif x[i] <= -1:
            x[i] = -1
    indice = (x*1/Q+1/2)
    xq = Q*floor(indice)
    return xq

def uniform_midrise_quantizer(x,n):
    x = copy(x)
    n = copy(n)
    m = size(x,axis=0)
    Q = (1-(-1))/(2**n)
    for i in range(0,m):
        if x[i] >= (1-Q):
            x[i] = 1
        elif x[i] <= -1:
            x[i] = -1
    indice = x*1/Q
    xq = Q*(floor(indice)+1/2)
    return xq

def convoluir(nx,x,nh,h):
    K1 = min(nx)
    N1 = min(nh)
    K2 = K1+size(x)-1
    N2 = N1+size(h)-1
    N = N2 - N1
    K = K2 - K1
    Y = N + K + 1
    y = zeros(Y)
    [nhi,hi] = ist(nh,h)
    nhi = nhi + K1 + N1
    ny = arange(K1+N1,K2+N2+1)
    xm = append(zeros(N),x)
    hmi = append(hi,zeros(K))
    y[0] = sum(xm*hmi)
    for n in range(1,Y):
        [nhm,hm] = meudeslocamento(hmi,nhi,n,'sim')  
        if n <= K:
            hm = hm[0:int(-n)]
        else:
            hm = hm[0:int(-n)] 
        y[n] = sum(xm*hm)
    return ny,y

def plotconvoluir(nx,x,nh,h,ny,y):
    clf()    
    stem(nx,x,linefmt='-.''r')
    show()
    grid
    stem(nh,h,'k')
    show()
    stem(ny,y,linefmt =':''b')
    axis([min(min(nh),min(nx),min(nh))-1,max(max(nh),max(nx), \
    max(ny))+1,min(min(h),min(x),min(y))-1,max(max(h),max(x),max(y))+1])
    xlabel('n',fontsize=18)
    ylabel('f[n]',fontsize=18)
    title('x[n] Vermelho, h[n] Preto, y[n] Azul',fontsize=18)

def compconv(nx,x,nh,h,ny,y):
    clf()
    stem(ny,y,linefmt ='--''r')  
    show()
    a = convolve(x,h)
    n = arange(min(nh)+min(nx),max(nh)+max(nx)+1)
    stem(n,a,linefmt ='-.''k')
    
    axis([min(min(nh),min(nx),min(ny))-1, \
    max(max(nh),max(nx),max(ny))+1,min(min(h),min(x),min(y))-1, \
    max(max(h),max(x),max(y))+1])
    
    xlabel('n',fontsize=18)
    ylabel('y[n]',fontsize=18)
    title('Comparação entre meu algoritmo de convolução em \
    vermelho com o do Python em preto',fontsize=12)
    
def ist(n,x):
    n = -n
    n = n[::-1]
    x = x[::-1]
    return n,x

def sound(x, fs):
    x = float32(x/amax(abs(x)))
    p = PyAudio()
    stream = p.open(format=paFloat32, channels=1, rate=fs, output=True)
    stream.write(x, num_frames=size(x))
    
    stream.stop_stream()
    stream.close()
    p.terminate()
    return

def leitor_audio_mono(a):
    fs, x = read(a)
    x = float32(x)
    x = float32((x[:,0] + x[:,1])/2) # Transforma Stereo em Mono
    sound(x,fs)
    
def eco(atraso,b0,musica):
    fs, x = read(musica)
    D = int(atraso*fs)
    x = float32(x)
    x = float32((x[:,0] + x[:,1])/2)
    y = lfilter(append(1,append(zeros(D-1),b0)),1.0,x)
    sound(y,fs)
    
def reverberar(atraso,a1,musica):
    fs, x = read(musica)
    D = int(atraso*fs)
    x = float32(x)
    x = float32((x[:,0] + x[:,1])/2)
    y = lfilter(append(1,zeros(D)),append(1,append(zeros(D-1),-a1)),x)
    sound(y,fs)

def eco_reverberar(atraso,b0,a1,musica):
    fs, x = read(musica)
    D = int(atraso*fs)
    a = append(2,append(zeros(D-1),-a1))
    b = append(2,append(zeros(D-1),b0))
    x = float32(x)
    x = float32((x[:,0] + x[:,1])/2)
    y = lfilter(b,a,x)
    sound(y,fs)
    
def filtroflangesin(musica,atraso,f,b0):
    fs, x = read(musica)
    x = float32(x)  
    N = size(x)
    n = arange(0,N)    
    D = int(atraso*fs)
    d = D*abs(sin(2*pi*n*f/fs))    
    x1 = append(x,zeros(D))
    y = zeros(N)    
    for i in range(0,N):
        y[i] = x[i] + x1[int(i - d[i])]    
    sound(y,fs)    
    return y,fs

def filtroflangedenteserra(musica,atraso,f,b0):
    fs, x = read(musica)
    x = float32(x)  
    N = size(x)
    n = arange(0,N)    
    D = int(atraso*fs)
    d = D*abs(sawtooth(n,(f/fs)))    
    x1 = append(x,zeros(D))
    y = zeros(N)    
    for i in range(0,N):
        y[i] = x[i] + x1[int(i - d[i])]    
    sound(y,fs)    
    return y,fs   

### EXERCÍCIO 1: Efeito de Eco (ou Delay)

O efeito de eco acontece quando um sinal de áudio é refletido em alguma parede ou obstáculo, retornando com um certo atraso e atenuação. Matematicamente, podemos representar esse efeito como sendo:

\begin{equation}
\large
    y[n] = x[n] + b_0x[n-D],
\end{equation}

em que b0 representa a atenuação recebida pelo sinal de eco e D o atraso entre o sinal original e seu retorno. A Figura 1 ilustra o diagrama em blocos desse efeito.

<center>![Figura 1: Filtro de Eco (delay).](fig1.png "ola")</center>

**a)** Implemente o efeito de eco utilizando a função lfilter do Python.

**b)** Utilize o audio1.wav como entrada $(x[n])$ e gere a saída $y[n]$ considerando um atraso de $330 ms$ e ganho $b0 = 0.8.$

**Música original:**

In [7]:
musica = 'musica.wav'
leitor_audio_mono(musica)

**Música alterada:**

In [8]:
atraso = 0.330
b0 = 0.8
musica = 'musica.wav'

eco(atraso,b0,musica)

### EXERCÍCIO 2: Efeito de Reverberação (Reverb) ###

O efeito de reverberação é semelhante ao eco, porém múltiplas reflexões do sinal são combinadas
no ponto de destino. A Figura 2 (à esquerda) ilustra este efeito. Do ponto de vista matemático, podemos representar esse efeito como sendo:

<center>![Figura 2: Filtro de Reverb.](fig2.png "ola")</center>

\begin{equation}
\large
    y[n] = a_1y[n-D] + x[n],
\end{equation}

em que $a_1$ representa a atenuação recebida pelas múltiplas reflexões. A Figura 2 (à direita) ilustra
o diagrama em blocos desse efeito.

$a)$ Implemente o efeito de reverb utilizando a função lfilter do Python.

$b)$ Utilize o audio1.wav como entrada $(x[n])$ e gere a saída $y[n]$ considerando um atraso de $330 ms$ e ganho $a1 = 0.8.$

**Música original:**

In [10]:
musica = 'audio1.wav'
leitor_audio_mono(musica)

**Música alterada:**

In [12]:
atraso = 0.330
a1 = 0.8
musica = 'audio1.wav'
 
reverberar(atraso,a1,musica)

**c)** Qual a diferença entre este efeito e o Eco do Exercício 1?

**Resposta:**

É possível perceber que no efeito de eco, as reflexões no som chegam ao emissor do som, enquanto no efeito de reverberar, os sons captados pelo recepetor é um resultado do som do emissor somado às suas reflexões


Experimente também suas funções de
eco e reverb utilizando o audio2.wav considerando um atraso de $300 ms$ e ganhos $b0 = a1 = 0.8$ para ajudar a tirar conclusões.

**Música original:**

In [13]:
musica = 'audio2.wav'

leitor_audio_mono(musica)

**Música alterada:**

In [14]:
musica = 'audio2.wav'
atraso = 0.300
b0 = 0.8
a1 = 0.8

eco(atraso,b0,musica)
reverberar(atraso,a1,musica)

**d)** Para criação de efeitos sonoros intencionais (e.g., em edição de áudio), Delay e Reverb podem
ser combinados para formar um efeito de maior preenchimento. Combine os efeitos de Delay e
Reverb em um único filtro IIR utilizando o audio2.wav como entrada considerando um atraso
de $300 ms$ e ganhos $b0 = 0.8 e a1 = 0.5.$

In [15]:
musica = 'audio2.wav'
atraso = 0.300
b0 = 0.8
a1 = 0.5

eco_reverberar(atraso,b0,a1,musica)

### EXERCÍCIO 3: Efeito Flanger###

O efeito flanger é um filtro de Eco, porém com atraso variável no tempo. Seu diagrama em
blocos está ilustrado pela Figura 3.

<center>![Figura 3: Filtro Flanger.](fig3.png "ola")</center>

A equação diferenças desses filtro é dada por:

\begin{equation}
\large
    y[n] = x[n] + b_0x[n-d(n)],
\end{equation}

Em que $d(n)$ é a função do atraso que varia com o tempo $n$. Comumente, $d(n)$ é uma função
senoidal ou dente de serra. Na sua forma senoidal, podemos escrevê-la como:

\begin{equation}
\large
    d(n) = D\left|\sin(\frac{2\pi{n}f}{f_s})\right|,
\end{equation}

em que $D$ é o atraso máximo, $f$ deve ser uma frequência de oscilação bastante baixa (de até $1 Hz$) e $fs$ é a frequência de amostragem. Note que, dessa forma, o atraso $0 \le d \le D$ varia de maneira senoidal no tempo.

**a)** Implemente o efeito de flanger utilizando o Python. Utilize o audio3.wav como entrada $(x[n])$
e gere a saída $y[n]$ considerando um atraso máximo de $5 ms$, frequência $f = 0.4 Hz$ e ganho
$b0 = 0.8.$

**Áudio original:**


In [16]:
fs, x = read('audio3.wav')
x = float32(x)
sound(x,fs)

**Áudio alterado:**

In [17]:
musica = 'audio3.wav'
atraso = 0.005
f = 0.4
b0 = 0.8

filtroflangesin(musica,atraso,f,b0)

(array([14., 17., 26., ..., -2., 10., -5.]), 96000)

**b)** Experimente trocar o atraso senoidal por uma função dente de serra e observe as diferenças.

**Obs.: Utilize a função sawtooth.**

In [18]:
musica = 'audio3.wav'
atraso = 0.005
f = 0.4
b0 = 0.8

filtroflangedenteserra(musica,atraso,f,b0)

(array([ 7., 10., 16., ..., -2., -4., 13.]), 96000)